
# הקדמה

**שם הסטודנט:** <!-- כתוב כאן את שמך כפי שמופיע במודל -->  
**אות ראשונה של שם המשפחה:** <!-- כתוב כאן -->  
**ארבע ספרות אחרונות של תעודת זהות:** <!-- כתוב כאן XXXX -->  

בתרגיל זה אנו עוסקים בבעיה של **למידה מונחית מסוג סיווג (Classification)**.  
המטרה היא לחזות האם נוסע בטיטאניק שרד (`Survived = 1`) או לא שרד (`Survived = 0`).  
מדד האיכות שבו נשתמש הוא **F1-score** לבעיה בינארית.



### שימוש ב־AI
במהלך הכנת המטלה נעזרתי בכלי AI (כגון ChatGPT) לצורך ניסוח, ארגון קוד והסברים.  
כל ההרצות, הניסויים והבחירות המודליות מוצגות במחברת זו.



## ספריות ושגרות עזר
נייבא ספריות לעיבוד נתונים, ויזואליזציה ומודלים. נגדיר פונקציות עזר לשכפול קוד.


In [ ]:

# ספריות בסיסיות
import pandas as pd
import numpy as np

# ויזואליזציה
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import f1_score, classification_report, ConfusionMatrixDisplay
from sklearn.impute import SimpleImputer

# אלגוריתמים
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# תצוגה
pd.set_option('display.max_columns', 100)
sns.set_theme()
plt.rcParams['figure.figsize'] = (6,4)

def plot_hist(series, title):
    plt.figure()
    series.dropna().hist(bins=30)
    plt.title(title)
    plt.xlabel(series.name)
    plt.ylabel('Count')
    plt.show()



## טעינת הנתונים
נשתמש בקבצים `titanic_train.csv` ו־`titanic_test.csv` מתוך התיקייה הנוכחית.


In [ ]:

train_path = 'titanic_train.csv'
test_path  = 'titanic_test.csv'

df_train = pd.read_csv(train_path)
df_test  = pd.read_csv(test_path)

print('Train shape:', df_train.shape)
print('Test shape :', df_test.shape)

df_train.head()


In [ ]:

# מידע כללי וערכים חסרים
display(df_train.info())
display(df_train.isna().sum().sort_values(ascending=False).to_frame('missing_train'))
display(df_test.isna().sum().sort_values(ascending=False).to_frame('missing_test'))



## ניתוח ראשוני (EDA)
נציג לפחות 4 ויזואליזציות: התפלגויות בסיסיות וטבלאות הישרדות לפי מאפיינים.


In [ ]:

# התפלגויות בסיסיות
if 'Age' in df_train.columns:
    plot_hist(df_train['Age'], 'התפלגות גיל (Train)')
if 'Fare' in df_train.columns:
    plot_hist(df_train['Fare'], 'התפלגות מחיר כרטיס (Train)')


In [ ]:

# שיעור הישרדות לפי מגדר ומחלקה (אם קיימים)
eda_tbls = {}
if {'Sex','Survived'}.issubset(df_train.columns):
    eda_tbls['Survival by Sex'] = df_train.pivot_table(index='Sex', values='Survived', aggfunc='mean')
if {'Pclass','Survived'}.issubset(df_train.columns):
    eda_tbls['Survival by Pclass'] = df_train.pivot_table(index='Pclass', values='Survived', aggfunc='mean')
for k, v in eda_tbls.items():
    print(k)
    display(v)


In [ ]:

# ויזואליזציות נוספות (לפחות שתיים)
if {'Survived','Age'}.issubset(df_train.columns):
    plt.figure()
    sns.boxplot(x="Survived", y="Age", data=df_train)
    plt.title("התפלגות גיל לפי הישרדות")
    plt.show()

if {'Pclass','Fare'}.issubset(df_train.columns):
    plt.figure()
    sns.boxplot(x="Pclass", y="Fare", data=df_train)
    plt.title("התפלגות מחיר כרטיס לפי מחלקת הפלגה")
    plt.show()



## הנדסת מאפיינים (Feature Engineering)
ניצור מאפיינים חדשים (`FamilySize`, `IsAlone`, `Deck`) ונגדיר עיבוד מקדים עבור נומרי/קטגוריאלי.


In [ ]:

def add_engineered_features(df):
    df = df.copy()
    # גודל משפחה
    if set(['SibSp','Parch']).issubset(df.columns):
        df['FamilySize'] = df['SibSp'].fillna(0) + df['Parch'].fillna(0) + 1
        df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    # סיפון מתוך Cabin
    if 'Cabin' in df.columns:
        df['Deck'] = df['Cabin'].astype(str).str[0].replace('n', np.nan)
    return df

df_train_fe = add_engineered_features(df_train)
df_test_fe  = add_engineered_features(df_test)

df_train_fe.head()



## הכנת צינור עיבוד (Preprocessing) ומודלים + GridSearchCV (5-Fold)


In [ ]:

target_col = 'Survived'
X = df_train_fe.drop(columns=[target_col])
y = df_train_fe[target_col]

# עמודות נומריות וקטגוריאליות
numeric_features = [c for c in X.columns if np.issubdtype(X[c].dtype, np.number)]
categorical_features = [c for c in X.columns if c not in numeric_features]

# הסרת מזהים
for col in ['PassengerId','Ticket']:
    if col in numeric_features:
        numeric_features.remove(col)
    if col in categorical_features:
        categorical_features.remove(col)

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# מודלים
pipe_logreg = Pipeline(steps=[('preprocess', preprocess),
                              ('model', LogisticRegression(max_iter=1000))])

pipe_tree = Pipeline(steps=[('preprocess', preprocess),
                            ('model', DecisionTreeClassifier(random_state=42))])

pipe_rf = Pipeline(steps=[('preprocess', preprocess),
                          ('model', RandomForestClassifier(random_state=42))])

pipe_knn = Pipeline(steps=[('preprocess', preprocess),
                           ('model', KNeighborsClassifier())])

# גרידים
param_grid_logreg = {'model__C':[0.1,1.0,3.0],'model__penalty':['l2'],'model__solver':['lbfgs','liblinear']}
param_grid_tree   = {'model__max_depth':[3,5,8,None],'model__min_samples_split':[2,5,10]}
param_grid_rf     = {'model__n_estimators':[100,300],'model__max_depth':[None,5,8],'model__min_samples_split':[2,5]}
param_grid_knn    = {'model__n_neighbors':[3,5,7,9],'model__weights':['uniform','distance']}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grids = [
    ('LogisticRegression', pipe_logreg, param_grid_logreg),
    ('DecisionTree',       pipe_tree,   param_grid_tree),
    ('RandomForest',       pipe_rf,     param_grid_rf),
    ('KNN',                pipe_knn,    param_grid_knn),
]

results = []
best_estimators = {}

for name, pipe, grid in grids:
    gs = GridSearchCV(estimator=pipe, param_grid=grid, scoring='f1', cv=cv, n_jobs=-1, refit=True, verbose=0)
    gs.fit(X, y)
    results.append({'model':name, 'best_score_mean_cv_f1':gs.best_score_, 'best_params':gs.best_params_})
    best_estimators[name] = gs.best_estimator_

res_df = pd.DataFrame(results).sort_values('best_score_mean_cv_f1', ascending=False)
res_df



## בחירת המודל הטוב ביותר והערכת Train


In [ ]:

best_row = res_df.iloc[0]
best_name = best_row['model']
best_model = best_estimators[best_name]

print('המודל הטוב ביותר:', best_name)
print('F1 ממוצע (CV):', best_row['best_score_mean_cv_f1'])
print('היפר-פרמטרים:', best_row['best_params'])

y_pred_train = best_model.predict(X)
print(classification_report(y, y_pred_train, digits=4))



## חיזוי על Test ויצוא תוצאות


In [ ]:

X_test = df_test_fe.copy()

pid = X_test['PassengerId'] if 'PassengerId' in X_test.columns else pd.Series(range(1, len(X_test)+1))

# התאמת עמודות
missing_cols = [c for c in X.columns if c not in X_test.columns]
for c in missing_cols:
    X_test[c] = np.nan
X_test = X_test[X.columns]

test_pred = best_model.predict(X_test)

sub = pd.DataFrame({'PassengerId': pid, 'Survived': test_pred.astype(int)})
sub.to_csv('titanic_submission.csv', index=False)

print("חמשת התחזיות הראשונות על קבוצת הבדיקה:")
print(sub.head())



## (רשות) מטריצת בלבול (Train)


In [ ]:

try:
    fig, ax = plt.subplots()
    ConfusionMatrixDisplay.from_predictions(y, y_pred_train, ax=ax)
    ax.set_title('Confusion Matrix (Train)')
    plt.show()
except Exception as e:
    print('לא ניתן לצייר מטריצה:', e)



## סיכום
הצגנו זרימת ML מלאה ל־Titanic: טעינה, EDA (4+ ויזואליזציות), הנדסת מאפיינים, ניסויים עם GridSearchCV ו־5-Fold (כולל KNN), בחירת מודל, וחיזוי על Test עם יצוא `titanic_submission.csv`.
